# Observations liées à la biodiversité dans les différents secteurs de la ville de Gatineau

L'outil ci-dessous permet de visualiser et de résumer les observations venant de iNaturalist (iNaturalist.ca) qui se trouvent sur le territoire de la ville de Gatineau.

Il a été élaboré par Enzo V. Motard, étudiant en sciences de la nature au Cégep de l'Outaouais, sous la supervision d'Olivier Rousseau et de Jean-René Hickey.

## Comment l'utiliser

1. Ci-dessous, sous le titre "L'outil", cliquer sur le triangle.

2. choisir un district électoral de la ville de Gatineau

3. Des résumés des observation apparaîtront : nombre d'espèces, types d'espèces, espèces menacées et espèces invasives présentes.

4. Sous les résumés, il y aura une carte avec les positions des observations
- En rouge : les espèces menacées. Amener le curseur au-dessus de l'observation pour voir le nom et le niveau de menace.
- En bleu : les espèces invasives.

Il arrive que la carte ne s'affiche pas correctement. Simplement faire raffraîchir la age, le problème devrait se régler.

## Détails méthodologiques

Les données proviennent de iNaturalist.ca, mais elles ont été quelques peu traitées et enrichies. Voici les principales manipulations qui ont été faites:

1. Seulement les données de qualité "recherche" ont été considérées. Ces données ont été validées par au moins deux personnes, ce qui les rend plus fiables. **Ajouter date**.

2. Certains doublons ont été éliminés. Pour le faire, nous fusionné les observations végétales qui ont été faites à un même endroit, à des moments différents.

3. Les données ont été enrichies de l'information sur les espèces envahissantes du Québec. [Source](https://www.quebec.ca/agriculture-environnement-et-ressources-naturelles/faune/gestion-faune-habitats-fauniques/gestion-especes-exotiques-envahissantes-animales/liste-especes).  

4. Les données ont été enrichies de l'information sur les espèces menacées du Québec. [Source](https://www.quebec.ca/gouvernement/gouvernement-ouvert/transparence-performance/indicateurs-statistiques/donnees-especes-situation-precaire).  



## L'outil

In [ ]:


import geopandas as gpd
import pandas as pd
from shapely import wkb
import folium
from ipywidgets import interact, widgets
import requests
from IPython.display import Javascript, display

# Set a large maximum height to prevent scrolling for most outputs
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))


# Aller chercher les données
def chercher_donnees(SQL):
    url = 'https://www.cogeomatique.ca/projet_inaturalist/01_select_data.php'
    SQL_requete = {"sql": SQL}

    response = requests.post(url, data=SQL_requete)

    if response.status_code == 200:
        json_data = response.json()
        if json_data.get("success"):
            return pd.DataFrame(json_data["rows"])
        else:
            print("Erreur dans la requête SQL.")
    else:
        print("Erreur serveur :", response.status_code)
    return pd.DataFrame()

# Requêtes
SQL_obs = """
SELECT id, scientific_name, common_name, longitude, latitude,
       iconic_taxon_name, menace, niveau_menace, invasive
FROM inaturalist_outaouais
"""

SQL_geom = "SELECT * FROM public.district_electoraux;"

# Charger les données
df_points = chercher_donnees(SQL_obs)
df_districts = chercher_donnees(SQL_geom)


# Corriger état menace / invasive
df_points["menace"] = df_points["menace"].astype(str).isin(["True"])
df_points["invasive"] = df_points["invasive"].astype(str).isin(["True"])

# Conversion des géométries WKB
df_districts["geom"] = df_districts["geom"].apply(
    lambda x: wkb.loads(bytes.fromhex(x))
)

gdf_districts = gpd.GeoDataFrame(
    df_districts,
    geometry="geom",
    crs="EPSG:4326"
)

# Conversion des points
gdf_points = gpd.GeoDataFrame(
    df_points,
    geometry=gpd.points_from_xy(df_points["longitude"], df_points["latitude"]),
    crs="EPSG:4326"
)

# Assigner chaque point à son district
points_dans_districts = gpd.sjoin(
    gdf_points,
    gdf_districts[["district", "geom"]],
    predicate="within",
    how="left"
)

# Fonction pour générer le rapport + carte
def rapport_par_district(nom_district):

    subset = points_dans_districts[points_dans_districts["district"] == nom_district]

    print(f"District : {nom_district}")
    print(f"Nombre total d'observations : {subset.shape[0]}")
    print(f"Nombre d'espèces observées : {subset['scientific_name'].nunique()}")

# Par taxon
    type_stats = (
    subset.groupby("iconic_taxon_name")
    .agg(
        observations=("scientific_name", "count"),
        especes=("scientific_name", pd.Series.nunique)
    )
    .reset_index()
    .rename(columns={"iconic_taxon_name": "taxon"})
    .sort_values(by="observations", ascending=False)
)

    print("\nObservations par taxon :")
    print(type_stats.to_string(index=False))

# Invasives
    invasives = subset[subset["invasive"] == True]
    nb_invasives = invasives["scientific_name"].nunique()
    print(f"\nNombre d'espèces invasives : {nb_invasives}")
    if nb_invasives > 0:
        liste_invasives = invasives["scientific_name"].value_counts().reset_index()
        liste_invasives.columns = ["espèce", "observations"]
        print("\nEspèces invasives observées :")
        print(liste_invasives.to_string(index=False))

# Menacées
    menacees = subset[subset["menace"] == True]
    nb_menacees = menacees["scientific_name"].nunique()
    print(f"\nNombre d'espèces menacées : {nb_menacees}")
    if nb_menacees > 0:
        liste_menacees = menacees["scientific_name"].value_counts().reset_index()
        liste_menacees.columns = ["espèce", "observations"]
        print("\nEspèces menacées observées :")
        print(liste_menacees.to_string(index=False))
        #return liste_menacees

    # Carte interactive
    district_geom = gdf_districts[gdf_districts["district"] == nom_district]
    centroid = district_geom.geometry.centroid.iloc[0]
    m = folium.Map(location=[centroid.y, centroid.x], zoom_start=14)

    # Polygone du district
    folium.GeoJson(
        district_geom,
        name=f"District {nom_district}",
        style_function=lambda x: {"color": "black", "weight": 2, "fillOpacity": 0.1}
    ).add_to(m)

    # Points
    for _, row in subset.iterrows():

        # Couleur
        color = (
            "red" if row["menace"] else
            ("blue" if row["invasive"] else "green")
        )

        # Hover data
        popup_lines = [
            f"<b>{row['scientific_name']}</b>",
            f"Nom commun : {row['common_name']}",
            f"Taxon : {row['iconic_taxon_name']}",
        ]

        if row["menace"]:
            popup_lines.append(f"Menacée")
            popup_lines.append(f"Niveau : {row['niveau_menace']}")

        if row["invasive"]:
            popup_lines.append("Invasive")

        popup_lines.append(f"Coordonnées : {row['latitude']}, {row['longitude']}")
        popup_lines.append(f"ID : {row['id']}")

        popup_html = "<br>".join(popup_lines)

        # Ajouter point
        folium.CircleMarker(
            location=[row["latitude"], row["longitude"]],
            radius=4,
            color=color,
            fill=True,
            fill_opacity=0.7,
            popup=popup_html
        ).add_to(m)

    # Légende
    legend_html = """
     <div style="
         position: fixed;
         bottom: 30px;
         left: 30px;
         width: 180px;
         height: 120px;
         background-color: white;
         border:2px solid grey;
         z-index:9999;
         font-size:14px;
         padding: 10px;
     ">
     <b>Légende</b><br>
     <i style="background:red; width:12px; height:12px; float:left; margin-right:8px; border-radius:50%;"></i> Menacée<br>
     <i style="background:blue; width:12px; height:12px; float:left; margin-right:8px; border-radius:50%;"></i> Invasive<br>
     <i style="background:green; width:12px; height:12px; float:left; margin-right:8px; border-radius:50%;"></i> Rien à noter
     </div>
     """
    m.get_root().html.add_child(folium.Element(legend_html))

    # Titre de la carte
    titre_html = f"""
     <h3 style="
         position: fixed;
         top: 10px;
         left: 50%;
         transform: translateX(-50%);
         z-index: 9999;
         background-color: rgba(255, 255, 255, 0.8);
         padding: 8px 15px;
         border-radius: 8px;
         border: 1px solid grey;
         font-size: 20px;
     ">
     Observations pour le district : {nom_district}
     </h3>
    """

    m.get_root().html.add_child(folium.Element(titre_html))

    return m

# Menu interactif
districts_disponibles = sorted(gdf_districts["district"].unique())

interact(
    rapport_par_district,
    nom_district=widgets.Dropdown(
        options=districts_disponibles,
        description="District :",
        layout=widgets.Layout(width="400px")
    )
)

<IPython.core.display.Javascript object>

interactive(children=(Dropdown(description='District :', layout=Layout(width='400px'), options=('Mitigomijokan…

<function __main__.rapport_par_district(nom_district)>